# Random forest 

### Import statements 

In [2]:
# Import necessary libraries
import json
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, auc, precision_recall_curve, confusion_matrix
import pandas as pd
import logging
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import confusion_matrix as cm
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

### Load data 

In [25]:
SMOTE_merged_df = pd.read_csv('/Users/dionnespaltman/Desktop/V3/train:test:val/new_merged_data.csv', sep=',')


In [27]:
with open('/Users/dionnespaltman/Desktop/V3/columns_action_units.json', 'r') as f:
    columns_action_units = json.load(f)

print(len(columns_action_units))
print(columns_action_units)

columns_au_12 = columns_action_units
# columns_au_12.append('VVR_1')
# columns_au_12.append('VVR_2')
print(columns_au_12)

119
['AU01_r__sum_values', 'AU01_r__variance', 'AU01_r__standard_deviation', 'AU01_r__maximum', 'AU01_r__minimum', 'AU01_r__mean', 'AU01_r__mean_abs_change', 'AU02_r__sum_values', 'AU02_r__variance', 'AU02_r__standard_deviation', 'AU02_r__maximum', 'AU02_r__minimum', 'AU02_r__mean', 'AU02_r__mean_abs_change', 'AU04_r__sum_values', 'AU04_r__variance', 'AU04_r__standard_deviation', 'AU04_r__maximum', 'AU04_r__minimum', 'AU04_r__mean', 'AU04_r__mean_abs_change', 'AU05_r__sum_values', 'AU05_r__variance', 'AU05_r__standard_deviation', 'AU05_r__maximum', 'AU05_r__minimum', 'AU05_r__mean', 'AU05_r__mean_abs_change', 'AU06_r__sum_values', 'AU06_r__variance', 'AU06_r__standard_deviation', 'AU06_r__maximum', 'AU06_r__minimum', 'AU06_r__mean', 'AU06_r__mean_abs_change', 'AU07_r__sum_values', 'AU07_r__variance', 'AU07_r__standard_deviation', 'AU07_r__maximum', 'AU07_r__minimum', 'AU07_r__mean', 'AU07_r__mean_abs_change', 'AU09_r__sum_values', 'AU09_r__variance', 'AU09_r__standard_deviation', 'AU09

In [28]:
from sklearn.model_selection import train_test_split, StratifiedKFold

X = columns_au_12
y = 'VVR_group'

# Step 1: Split data into training and holdout test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Step 2: Apply 5-fold cross-validation on the training data with stratification
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_val, y_train_val)):
    print(f"Fold {fold+1}:")
    X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
    y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]
    
    # Train and validate your model on X_train, y_train, X_val, y_val


InvalidParameterError: The 'stratify' parameter of train_test_split must be an array-like or None. Got 'VVR_group' instead.

In [3]:
# # Open the JSON file and load its contents
# train = pd.read_json('/Users/dionnespaltman/Desktop/V3/train:test:val/train.json')
# test = pd.read_json('/Users/dionnespaltman/Desktop/V3/train:test:val/test.json')
# val = pd.read_json('/Users/dionnespaltman/Desktop/V3/train:test:val/val.json')

### Train 

In [24]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Define features and target
X_train = train.drop('VVR_group', axis=1)
y_train = train['VVR_group'].values

print(len(X_train))

# Define and train RandomForest model
rf = make_pipeline(RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=0))
rf.fit(X_train, y_train)


102


Pipeline(steps=[('randomforestclassifier',
                 RandomForestClassifier(n_estimators=200, n_jobs=-1,
                                        random_state=0))])

### Hyperparameter tuning using val set 

In [20]:
from sklearn.model_selection import GridSearchCV

# Define features and target for validation set
X_val = val.drop('VVR_group', axis=1)
y_val = val['VVR_group'].values

# Define the parameter grid for RandomForest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [1, 4],
    # 'max_features': ['auto', 'sqrt', 'log2'],
    # 'max_leaf_nodes': [None, 10, 20, 30],
    # 'min_impurity_decrease': [0.0, 0.1]
}

# Instantiate the random forest classifier
rf_tuned = RandomForestClassifier(random_state=0)

# Create the grid search object
# 5-fold cross validation
grid_search = GridSearchCV(estimator=rf_tuned, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters found
print("Best parameters found: ", grid_search.best_params_)

# Use the best estimator to make predictions
best_rf_tuned = grid_search.best_estimator_

# Predict on the validation set using the best estimator
pred_val_tuned = best_rf_tuned.predict(X_val)

# Calculate metrics
accuracy_val_tuned = accuracy_score(y_val, pred_val_tuned)
report_val_tuned = classification_report(y_val, pred_val_tuned)

# Print results
print(f"Accuracy on Validation Data (Tuned): {accuracy_val_tuned}")
print("Classification Report (Tuned):")
print(report_val_tuned)


Best parameters found:  {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
Accuracy on Validation Data (Tuned): 0.9117647058823529
Classification Report (Tuned):
              precision    recall  f1-score   support

           0       0.94      0.89      0.91        18
           1       0.88      0.94      0.91        16

    accuracy                           0.91        34
   macro avg       0.91      0.91      0.91        34
weighted avg       0.91      0.91      0.91        34



### Evaluate on test set 

In [23]:
# Define features and target for validation set
X_test = test.drop('VVR_group', axis=1)
y_test = test['VVR_group'].values

#Perform hyperparameter tuning on the validation set
# (GridSearchCV or other hyperparameter tuning technique)
best_params = {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}  # Example best parameters

# Train the model using the entire training set with the best parameters
best_rf = RandomForestClassifier(**best_params)
best_rf.fit(X_train, y_train)

# Evaluate the model on the test set
pred_test = best_rf.predict(X_test)
accuracy_test = accuracy_score(y_test, pred_test)
report_test = classification_report(y_test, pred_test)

# Print the results
print(f"Accuracy on Test Data: {accuracy_test}")
print("Classification Report on Test Data:")
print(report_test)

Accuracy on Test Data: 0.7941176470588235
Classification Report on Test Data:
              precision    recall  f1-score   support

           0       0.69      0.75      0.72        12
           1       0.86      0.82      0.84        22

    accuracy                           0.79        34
   macro avg       0.77      0.78      0.78        34
weighted avg       0.80      0.79      0.80        34

